***Important*** DO NOT CLEAR THE OUTPUT OF THIS NOTEBOOK AFTER EXECUTION!!!

In [1]:
!pip install -q google-cloud-storage==1.43.0
!pip install -q graphframes

In [2]:
# if the following command generates an error, you probably didn't enable 
# the cluster security option "Allow API access to all Google Cloud services"
# under Manage Security → Project Access when setting up the cluster
!gcloud dataproc clusters list --region us-central1

NAME          PLATFORM  PRIMARY_WORKER_COUNT  SECONDARY_WORKER_COUNT  STATUS   ZONE           SCHEDULED_DELETE
cluster-0016  GCE       2                                             RUNNING  us-central1-a


# Imports & Setup

In [3]:
import pyspark
import sys
from collections import Counter, OrderedDict, defaultdict
import itertools
from itertools import islice, count, groupby
import pandas as pd
import os
import re
from operator import itemgetter
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
from time import time
from pathlib import Path
import pickle
import pandas as pd
from google.cloud import storage

import hashlib
def _hash(s):
    return hashlib.blake2b(bytes(s, encoding='utf8'), digest_size=5).hexdigest()

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# if nothing prints here you forgot to include the initialization script when starting the cluster
!ls -l /usr/lib/spark/jars/graph*

-rw-r--r-- 1 root root 247882 Jan 20 13:33 /usr/lib/spark/jars/graphframes-0.8.2-spark3.1-s_2.12.jar


In [5]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf, SparkFiles
from pyspark.sql import SQLContext
from graphframes import *

In [6]:
spark

In [7]:
# Put your bucket name below and make sure you can access it without an error
bucket_name = 'hw3ir322' 
full_path = f"gs://{bucket_name}/"
paths=[]

client = storage.Client()
blobs = client.list_blobs(bucket_name)
for b in blobs:
    if "parquet" in b.name:
#         print(b.name)
        paths.append(full_path+b.name)

***GCP setup is complete!*** If you got here without any errors you've earned 10 out of the 35 points of this part.

# Building an inverted index

Here, we read the entire corpus to an rdd, directly from Google Storage Bucket and use your code from Colab to construct an inverted index.

In [8]:
parquetFile = spark.read.parquet(*paths)
doc_title_pairs = parquetFile.select("title", "id").rdd

We will count the number of pages to make sure we are looking at the entire corpus. The number of pages should be more than 6M

In [9]:
# Count number of wiki pages
parquetFile.count()

6348910

Let's import the inverted index module. Note that you need to use the staff-provided version called `inverted_index_gcp.py`, which contains helper functions to writing and reading the posting files similar to the Colab version, but with writing done to a Google Cloud Storage bucket.

In [10]:
# if nothing prints here you forgot to upload the file inverted_index_gcp.py to the home dir
%cd -q /home/dataproc
!ls inverted_index_gcp.py

inverted_index_gcp.py


In [11]:
# adding our python module to the cluster
sc.addFile("/home/dataproc/inverted_index_gcp.py")
sys.path.insert(0,SparkFiles.getRootDirectory())

In [12]:
from inverted_index_gcp import InvertedIndex

**YOUR TASK (10 POINTS)**: Use your implementation of `word_count`, `reduce_word_counts`, `calculate_df`, and `partition_postings_and_write` functions from Colab to build an inverted index for all of English Wikipedia in under 2 hours.

A few notes: 
1. The number of corpus stopwords below is a bit bigger than the colab version since we are working on the whole corpus and not just on one file.
2. You need to slightly modify your implementation of  `partition_postings_and_write` because the signature of `InvertedIndex.write_a_posting_list` has changed and now includes an additional argument called `bucket_name` for the target bucket. See the module for more details.
3. You are not allowed to change any of the code not coming from Colab. 

In [19]:
english_stopwords = frozenset(stopwords.words('english'))
corpus_stopwords = ['category', 'references', 'also”', 'links', 'extrenal',
                 'first', 'see', 'new', 'two', 'list', 'may', 'one', 'district',
                 'including', 'became', 'however', 'com', 'many', 'began',
                 'make', 'made', 'part', 'would', 'people', 'second', 'also',
                 'following', 'history', 'thumb', 'external']

all_stopwords = english_stopwords.union(corpus_stopwords)
RE_WORD = re.compile(r"""[\#\@\w](['\-]?\w){2,24}""", re.UNICODE)
stemmer = PorterStemmer()
NUM_BUCKETS = 124
def token2bucket_id(token):
  return int(_hash(token),16) % NUM_BUCKETS

# PLACE YOUR CODE HERE

def partition_postings_and_write(postings, base_dir):
  bucket_rdd = postings.map(lambda x: (token2bucket_id(x[0]), x)).groupByKey()

  def write_bucket(b_w_pl):
      bucket_id, word_posting_pairs = b_w_pl
      return InvertedIndex.write_a_posting_list((bucket_id, list(word_posting_pairs)),base_dir,bucket_name)

  posting_locs_list = bucket_rdd.map(write_bucket)

  return posting_locs_list


def reduce_word_counts(unsorted_pl):
  ''' Returns a sorted posting list by wiki_id.
  Parameters:
  -----------
    unsorted_pl: list of tuples
      A list of (wiki_id, tf) tuples 
  Returns:
  --------
    list of tuples
      A sorted posting list.
  '''
  # YOUR CODE HERE
  return sorted(unsorted_pl, key = lambda x: x[0])

def calculate_df(postings):
  ''' Takes a posting list RDD and calculate the df for each token.
  Parameters:
  -----------
    postings: RDD
      An RDD where each element is a (token, posting_list) pair.
  Returns:
  --------
    RDD
      An RDD where each element is a (token, df) pair.
  '''
  # YOUR CODE HERE
  return postings.map(lambda token: (token[0], len(token[1])))

def tokenize(text):
  stemmer = PorterStemmer()
  tokens = [token.group() for token in RE_WORD.finditer(text.lower())]
  stemmed = [stemmer.stem(token) for token in tokens if token not in all_stopwords]
  return stemmed



def stemmed_word_count(text, id):
  tokens = tokenize(text)
  token_counts = Counter(tokens)  # Calculate term frequencies efficiently
  res_list = [
        (token, (id, count))
        for token, count in token_counts.items()
        if token not in all_stopwords
    ]
  return res_list

In [14]:
# time the index creation time
# word counts map
word_counts = doc_title_pairs.flatMap(lambda x: stemmed_word_count(x[0], x[1]))
postings = word_counts.groupByKey().mapValues(reduce_word_counts)
w2df = calculate_df(postings)
w2df_dict = w2df.collectAsMap()



In [20]:
# partition posting lists and write out
_ = partition_postings_and_write(postings, "title").collect()

25/01/20 20:55:22 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 2.0 in stage 18.0 (TID 1034) (cluster-0016-w-0.c.ir-hw3-444616.internal executor 7): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/util.py", line 73, in wrapper
    return f(*args, **kwargs)
  File "/tmp/ipykernel_68096/4038539762.py", line 22, in write_bucket
  File "/hadoop/yarn/nm-local-dir/usercache/root/appcache/application_1737379950114_0009/container_1737379950114_0009_01_000008/inverted_index_gcp.py", line 182, in write_a_posting_list
    with clos

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 3 in stage 18.0 failed 4 times, most recent failure: Lost task 3.3 in stage 18.0 (TID 1055) (cluster-0016-w-1.c.ir-hw3-444616.internal executor 9): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 604, in main
    process()
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/worker.py", line 596, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "/usr/lib/spark/python/lib/pyspark.zip/pyspark/util.py", line 73, in wrapper
    return f(*args, **kwargs)
  File "/tmp/ipykernel_68096/4038539762.py", line 22, in write_bucket
  File "/hadoop/yarn/nm-local-dir/

25/01/20 20:55:25 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 9.1 in stage 18.0 (TID 1056) (cluster-0016-w-0.c.ir-hw3-444616.internal executor 7): TaskKilled (Stage cancelled)
25/01/20 20:55:26 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 10.0 in stage 18.0 (TID 1062) (cluster-0016-w-1.c.ir-hw3-444616.internal executor 10): TaskKilled (Stage cancelled)
25/01/20 20:55:26 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 5.3 in stage 18.0 (TID 1059) (cluster-0016-w-0.c.ir-hw3-444616.internal executor 7): TaskKilled (Stage cancelled)
25/01/20 20:55:26 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 7.3 in stage 18.0 (TID 1060) (cluster-0016-w-0.c.ir-hw3-444616.internal executor 8): TaskKilled (Stage cancelled)
25/01/20 20:55:26 WARN org.apache.spark.scheduler.TaskSetManager: Lost task 1.3 in stage 18.0 (TID 1061) (cluster-0016-w-0.c.ir-hw3-444616.internal executor 8): TaskKilled (Stage cancelled)


In [ ]:
# collect all posting lists locations into one super-set
super_posting_locs = defaultdict(list)
for blob in client.list_blobs(bucket_name, prefix='title'):
  if not blob.name.endswith("pickle"):
    continue
  with blob.open("rb") as f:
    posting_locs = pickle.load(f)
    for k, v in posting_locs.items():
      super_posting_locs[k].extend(v)

Putting it all together

In [ ]:
# Create inverted index instance
inverted = InvertedIndex()
# Adding the posting locations dictionary to the inverted index
inverted.posting_locs = super_posting_locs
# Add the token - df dictionary to the inverted index
inverted.df = w2df_dict
# write the global stats out
inverted.write_index('.', 'index')
# upload to gs
index_src = "index.pkl"
index_dst = f'gs://{bucket_name}/title_stemmed/{index_src}'
!gsutil cp $index_src $index_dst

In [ ]:
!gsutil ls -lh $index_dst